**Helper functions and installing libraries**

In [139]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date

In [165]:
#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    result = requests.get(api_call)
    result = result.json()

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False
        print(f'No noxt page for api call {api_call}')

    return result, has_more

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

**Get league names**

In [3]:
leagues = requests.get(f"https://api.sportmonks.com/v3/football/leagues?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep")

leagues = leagues.json()

leagues

{'data': [{'id': 8,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Premier League',
   'active': True,
   'short_code': 'UK PL',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/8/8.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-06 15:30:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 72,
   'sport_id': 1,
   'country_id': 38,
   'name': 'Eredivisie',
   'active': True,
   'short_code': 'NLD E',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/72.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-06 18:00:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 82,
   'sport_id': 1,
   'country_id': 11,
   'name': 'Bundesliga',
   'active': True,
   'short_code': 'GER BI',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/18/82.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2024-10-06 17:30:00',
   'category': 1,
   'has

In [4]:
wanted_leagues = ['Premier League', 'La Liga', 'Bundesliga', 'Serie A']
wanted_features = ['id', 'country_id', 'name', 'image_path']

leagues_df = {}

for league in leagues['data']:
    if league['name'] in wanted_leagues:
        for feature in wanted_features:
            app_dictionary(feature, league[f'{feature}'], leagues_df)
        
leagues_df = pd.DataFrame(leagues_df)
leagues_df

,id,country_id,name,image_path
0,8,462,Premier League,https://cdn.sportmonks.com/images/soccer/leagu...
1,82,11,Bundesliga,https://cdn.sportmonks.com/images/soccer/leagu...
2,384,251,Serie A,https://cdn.sportmonks.com/images/soccer/leagu...
3,564,32,La Liga,https://cdn.sportmonks.com/images/soccer/leagu...
4,648,5,Serie A,https://cdn.sportmonks.com/images/soccer/leagu...


**Get season ID by search**

In [63]:
season_names = ['2023']
season_data = {'data':[]}
league_id = leagues_df['id'].to_numpy()

for season_name in season_names:
    seasons, has_more = run_api(f"https://api.sportmonks.com/v3/football/seasons/search/{season_name}?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep", True)
    
    page = 2
    while has_more == True:
        next_seasons, has_more = run_api(f"https://api.sportmonks.com/v3/football/seasons/search/{season_name}?page={page}&api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep", True)
        seasons['data'] = seasons['data'] + next_seasons['data']
        page += 1
    
    for season in seasons['data']:
        if season['league_id'] in league_id:
            season_data['data'].append(season)

for season in season_data['data']:
    if season['league_id'] in league_id:
        print(season)

{'id': 19734, 'sport_id': 1, 'league_id': 8, 'tie_breaker_rule_id': 1526, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-05', 'ending_at': '2023-05-28', 'standings_recalculated_at': '2023-11-30 15:26:55', 'games_in_current_week': False}
{'id': 19744, 'sport_id': 1, 'league_id': 82, 'tie_breaker_rule_id': 171, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-05', 'ending_at': '2023-05-27', 'standings_recalculated_at': '2023-11-30 15:44:34', 'games_in_current_week': False}
{'id': 19799, 'sport_id': 1, 'league_id': 564, 'tie_breaker_rule_id': 170, 'name': '2022/2023', 'finished': True, 'pending': False, 'is_current': False, 'starting_at': '2022-08-12', 'ending_at': '2023-06-04', 'standings_recalculated_at': '2023-08-09 14:53:21', 'games_in_current_week': False}
{'id': 19806, 'sport_id': 1, 'league_id': 384, 'tie_breaker_rule_id': 170, 'name': '2022/2023', 'finished': True, 'pending'

In [64]:
seasonByLeagueID = pd.DataFrame(season_data['data'])
seasonByLeagueID

,id,sport_id,league_id,tie_breaker_rule_id,name,finished,pending,is_current,starting_at,ending_at,standings_recalculated_at,games_in_current_week
0,19734,1,8,1526,2022/2023,True,False,False,2022-08-05,2023-05-28,2023-11-30 15:26:55,False
1,19744,1,82,171,2022/2023,True,False,False,2022-08-05,2023-05-27,2023-11-30 15:44:34,False
2,19799,1,564,170,2022/2023,True,False,False,2022-08-12,2023-06-04,2023-08-09 14:53:21,False
3,19806,1,384,170,2022/2023,True,False,False,2022-08-13,2023-06-11,2023-11-30 15:27:30,False
4,21207,1,648,577,2023,True,False,False,2023-04-15,2023-12-07,2023-12-07 02:41:01,False
5,21646,1,8,171,2023/2024,True,False,False,2023-08-11,2024-05-19,2024-05-19 17:01:55,False
6,21694,1,564,170,2023/2024,True,False,False,2023-08-11,2024-05-26,2024-05-26 21:06:57,False
7,21795,1,82,1526,2023/2024,True,False,False,2023-08-18,2024-05-18,2024-05-18 15:36:03,False
8,21818,1,384,170,2023/2024,True,False,False,2023-08-19,2024-06-02,2024-06-03 00:06:31,False


Get Teams by Season ID

In [94]:
season_id = seasonByLeagueID['id'].to_numpy()
teams_data = {'data':[]}

for id in season_id:
    teams, has_more = run_api(f"https://api.sportmonks.com/v3/football/teams/seasons/{id}?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep")
    print(f'Season id: {id}')
    
    page = 2
    while has_more == True:
        next_teams, has_more = run_api(f"https://api.sportmonks.com/v3/football/teams/seasons/{id}?page={page}&api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep")
        teams['data'] = teams['data'] + next_teams['data']
        page += 1
    
    for team in teams['data']:
        team['season_id'] = id
        teams_data['data'].append(team)

An exception occurred for api call https://api.sportmonks.com/v3/football/teams/seasons/19734?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep
Season id: 19734
An exception occurred for api call https://api.sportmonks.com/v3/football/teams/seasons/19744?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep
Season id: 19744
An exception occurred for api call https://api.sportmonks.com/v3/football/teams/seasons/19799?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep
Season id: 19799
An exception occurred for api call https://api.sportmonks.com/v3/football/teams/seasons/19806?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep
Season id: 19806
An exception occurred for api call https://api.sportmonks.com/v3/football/teams/seasons/21207?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep
Season id: 21207
An exception occurred for api call https://api.sportmonks.com/v3/football/teams/seasons/21

In [97]:
teams_df = pd.DataFrame(teams_data['data'])
teams_df
teams_df.to_csv('data/teams2.csv')

In [122]:
teams_df['id'].unique()[-1]

585

**Get fixtures by team**

In [161]:
start_date = '2022-09-01'
end_date = '2022-10-01'

In [157]:
# Initial date (starting point)
start_date = datetime(2022, 8, 5)  # Example starting date in 'year-month-day'

# Number of intervals and days
days_interval = 100
total_days = 667

# Variable to store the previous date
previous_date = None

# Loop over the 300 days in 100-day intervals
for i in range(0, total_days, days_interval):
    current_date = start_date + timedelta(days=i)
    formatted_date = current_date.strftime('%Y-%m-%d')

    # Print the previous and current date
    if previous_date:
        print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
    else:
        print(f"First date: {formatted_date}")  # For the first date

    # Update the previous date for the next iteration
    previous_date = formatted_date


First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Previous date: 2022-11-13 -> Next date: 2023-02-21
Previous date: 2023-02-21 -> Next date: 2023-06-01
Previous date: 2023-06-01 -> Next date: 2023-09-09
Previous date: 2023-09-09 -> Next date: 2023-12-18
Previous date: 2023-12-18 -> Next date: 2024-03-27


Looping over periods of 100 days (limitation of the API)

In [163]:
# Initial date (starting point)
start_date = datetime(2022, 8, 5)  # Example starting date in 'year-month-day'

# Number of intervals and days
days_interval = 100
total_days = 667

# Variable to store the previous date
previous_date = None

In [164]:
teams_id = teams_df['id'].unique()
fixture_data = {'data': []}

# Loop over the 300 days in 100-day intervals
for i in range(0, total_days, days_interval):
    current_date = start_date + timedelta(days=i)
    formatted_date = current_date.strftime('%Y-%m-%d')

    # Print the previous and current date
    if previous_date:
        print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
        for id in teams_id:
            fixtures, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/between/{str(previous_date)}/{str(formatted_date)}/{id}?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep&per_page=50")
            print(f'Fixtures for team: {id}')

            page = 2
            while has_more == True:
                next_features, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/between/{str(previous_date)}/{str(formatted_date)}/{id}?page={page}&api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep&per_page=50")
                #fixtures['data'].append(i for i in next_features['data'])
                fixtures['data'] = fixtures['data'] + next_teams['data']
                page += 1
            
            try:
                for fixture in fixtures['data']:
                    fixture_data['data'].append(fixture)
            except:
                print(f'No data for team {id}')
    else:
        print(f"First date: {formatted_date}")  # For the first date

    # Update the previous date for the next iteration
    previous_date = formatted_date


First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
Fixtures for team: 78
Fixtures for team: 11
Fixtures for team: 8
Fixtures for team: 19
Fixtures for team: 42
Fixtures for team: 29
Fixtures for team: 71
Fixtures for team: 51
Fixtures for team: 14
Fixtures for team: 236
Fixtures for team: 52
Fixtures for team: 20
Fixtures for team: 63
Fixtures for team: 13
Fixtures for team: 9
Fixtures for team: 1
Fixtures for team: 65
Fixtures for team: 18
Fixtures for team: 15
Fixtures for team: 6
Fixtures for team: 3317
Fixtures for team: 1079
Fixtures for team: 510
Fixtures for team: 3321
Fixtures for team: 3543
Fixtures for team: 68
Fixtures for team: 82
Fixtures for team: 3320
Fixtures for team: 67
Fixtures for team: 277
Fixtures for team: 90
Fixtures for team: 683
Fixtures for team: 3319
Fixtures for team: 999
Fixtures for team: 2726
Fixtures for team: 366
Fixtures for team: 794
Fixtures for team: 503
Fixtures for team: 3477
Fixtures for team: 3468
Fixtures for team: 485
F

In [166]:
play_sound()

Saving as csv

In [167]:
fixture_data = pd.DataFrame(fixture_data['data'])

fixture_data.to_csv('data/fixture_data_full.csv')

**API call for events and add ons**

In [184]:
next_features, has_more = run_api(f"https://api.sportmonks.com/v3/football/fixtures/18535049?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21")

next_features

No noxt page for api call https://api.sportmonks.com/v3/football/fixtures/18535049?api_token=4Wduau6vViz1F329mVrSrahEuod53KHjzcYk43knfbtNwWjGzQchWSrrPDep&per_page=50&include=formations;weatherReport;referees;events:type_id,player_name,addition,minute&filters=eventTypes:10,19,20,21


{'data': {'id': 18535049,
  'sport_id': 1,
  'league_id': 8,
  'season_id': 19734,
  'stage_id': 77457864,
  'group_id': None,
  'aggregate_id': None,
  'round_id': 274668,
  'state_id': 5,
  'venue_id': 206,
  'name': 'Manchester United vs Brighton & Hove Albion',
  'starting_at': '2022-08-07 13:00:00',
  'result_info': 'Brighton & Hove Albion won after full-time.',
  'leg': '1/1',
  'details': None,
  'length': 90,
  'placeholder': False,
  'has_odds': True,
  'has_premium_odds': False,
  'starting_at_timestamp': 1659877200,
  'formations': [{'id': 336844,
    'fixture_id': 18535049,
    'participant_id': 78,
    'formation': '3-4-2-1',
    'location': 'away'},
   {'id': 336843,
    'fixture_id': 18535049,
    'participant_id': 14,
    'formation': '4-2-3-1',
    'location': 'home'}],
  'weatherreport': {'id': 103325,
   'fixture_id': 18535049,
   'venue_id': 206,
   'temperature': {'day': 20.95,
    'morning': 14.48,
    'evening': 18.84,
    'night': 14.99},
   'feels_like': {'day'